In [1]:
import numpy as np
import assist
from sorcha.ephemeris.simulation_geometry import equatorial_to_ecliptic
from sorcha.ephemeris.orbit_conversion_utilities import universal_keplerian
from astroquery.jplhorizons import Horizons

np.set_printoptions(precision=14, linewidth=200, suppress=True)

In [2]:
# dict of available planets and their corresponding SPICE IDs
# NOTE: because of the way the default planet ephems are set up, the
# body centers of each planet cannot be queried directly. instead, we
# have to use the planetary system barycenters
avail_planets = {
    "mercury": 199,
    "venus": 299,
    "earth": 399,
    "mars": 4,
    "jupiter": 5,
    "saturn": 6,
    "uranus": 7,
    "neptune": 8,
}

In [3]:
def get_planet_orbits(t_mjd_tdb):
    ### constants (other users will need to modify assist_data_path)
    # start by creating an assist ephem object for the planet ephemeris
    assist_data_path = "/Users/rahil/grad/random/assist/data"
    ephem = assist.Ephem(
        f"{assist_data_path}/linux_p1550p2650.440",
    )

    # compute the ephemeris query time (MJD -> JD -> ASSIST time)
    t_query = t_mjd_tdb + 2400000.5 - ephem.jd_ref
    sun_query = ephem.get_particle("sun", t_query)
    sun_state = np.array(sun_query.xyz + sun_query.vxyz)
    # get the orbits of the planets
    pl_orbits = {}
    # print("Orbits of the planets: sma[au], ecc, inc[radians], long. node.[radians], arg. peri.[radians], M[radians]")
    for body in avail_planets:
        # get heliocentric equatorial state vector
        body_query = ephem.get_particle(body, t_query)
        equ_state = np.array(body_query.xyz + body_query.vxyz) - sun_state
        # convert to heliocentric ecliptic state vector
        ecl_pos = equatorial_to_ecliptic(equ_state[:3])
        ecl_vel = equatorial_to_ecliptic(equ_state[3:])
        # convert to keplerian elements
        elems = np.array(universal_keplerian(sun_query.m, *ecl_pos, *ecl_vel, t_mjd_tdb))
        # wrap elems[2:6] to [0,2pi]
        elems[2:6] = np.mod(elems[2:6], 2 * np.pi)
        pl_orbits[body] = tuple(float(el) for el in elems)
    return pl_orbits

In [4]:
# pick a time (use J2000 for now)
t_mjd_tdb = 51544.5
# get the orbits of the planets
pl_orbits = get_planet_orbits(t_mjd_tdb)
# now get the same orbits from horizons
for body in avail_planets.keys():
    obj = Horizons(id=avail_planets[body], location="@sun", epochs=t_mjd_tdb + 2400000.5)
    elems = obj.elements(refsystem="J2000", refplane="ecliptic")
    keys = ("a", "e", "incl", "Omega", "w", "M")
    hrzns_elems = np.array([float(elems[k].data[0]) for k in keys])
    hrzns_elems[2:] *= np.pi / 180.0  # convert to radians
    print(f"Horizons orbit of {body}: {hrzns_elems}")
    print(f"Layup orbit of {body}: {pl_orbits[body]}\n")

(ASSIST) The JPL asteroid ephemeris file has not been found. Asteroid forces have been disabled.


Horizons orbit of mercury: [0.38709821218416 0.20563029337056 0.12226056374145 0.84352702699409 0.50831460719944 3.05076367583143]
Layup orbit of mercury: (0.387098254577587, 0.20563016180610422, 0.12226056374145156, 0.84352702699409, 0.508314656555072, 3.050763624960782)

Horizons orbit of venus: [0.72332692747965 0.00675578625189 0.05924676613057 1.33829009340838 0.96317682556011 0.87466777308793]
Layup orbit of venus: (0.7233287133824766, 0.006757353070801505, 0.059246766130575286, 1.3382900934083783, 0.9634571958675414, 0.8743873964358694)

Horizons orbit of earth: [1.00044882893419 0.01711862906747 0.00000729737182 2.35760328954778 5.70248317823    6.25905187588486]
Layup orbit of earth: (1.0004519384173047, 0.017121683034759975, 7.297386338709356e-06, 2.3576032945807794, 5.702478792047673, 6.259056257653747)

Horizons orbit of mars: [1.52367899229846 0.09331510156051 0.03228643490644 0.8650201965175  5.00102073557554 0.33783436841111]
Layup orbit of mars: (1.5236795776562282, 0.0